In [1]:
from PIL import Image
from tensorflow.keras.layers import Dense,Input
from keras.layers import Input , Lambda , Flatten , Dense , Dropout
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np 
import glob as glob
datagen = ImageDataGenerator()
import matplotlib.pyplot as plt

import json
import random
import cv2

import tensorflow as tf

D:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
IMAGE_SIZE = [224, 224]

train_photo = 'train_photo/'
test_photo = 'test_photo/'
val_photo = 'val_photo/'

In [3]:
myvgg = VGG16(input_shape=IMAGE_SIZE +  [3], weights='imagenet', include_top=False)

In [4]:
for layer in myvgg.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(myvgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [5]:
myfolders = glob.glob('Desktop\task4\train_photo')

In [6]:
def addTopmodel(bottom_model, num_classes, D=256):
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")( top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")( top_model)
    return top_model

myvgg.input
myvgg.summary

<bound method Network.summary of <keras.engine.training.Model object at 0x000001C7EBFF7108>>

In [7]:
num_classes = 2

This_fc = addTopmodel(myvgg, num_classes)

model = Model(inputs=myvgg.input, outputs=This_fc)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
model = Model(inputs=myvgg.input, output=This_fc)

D:\Users\asus\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [11]:
train_set = train_datagen.flow_from_directory('facereg/train_photo/',
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('facereg/test_photo/',
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical')

Found 998 images belonging to 2 classes.
Found 994 images belonging to 2 classes.


In [15]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)
nb_train_samples = 149
nb_validation_samples = 19 
epochs = 8
batch_size = 16
    

r = model.fit_generator(
       train_set,
      steps_per_epoch = nb_train_samples //batch_size,
      validation_data=test_set,
      epochs= epochs,
     
      validation_steps= nb_validation_samples //batch_size)
    
    

Epoch 1/8
9/9 [==============================] - 61s 7s/step - loss: 0.0278 - accuracy: 0.9896 - val_loss: 1.2121 - val_accuracy: 0.5938
Epoch 2/8
9/9 [==============================] - 60s 7s/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.5781 - val_accuracy: 0.7188
Epoch 3/8
9/9 [==============================] - 64s 7s/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 1.0168 - val_accuracy: 0.6562
Epoch 4/8
9/9 [==============================] - 66s 7s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2440 - val_accuracy: 0.8750
Epoch 5/8
9/9 [==============================] - 59s 7s/step - loss: 5.4859e-04 - accuracy: 1.0000 - val_loss: 0.3827 - val_accuracy: 0.9062
Epoch 6/8
9/9 [==============================] - 64s 7s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.5355 - val_accuracy: 0.8438
Epoch 7/8
9/9 [==============================] - 63s 7s/step - loss: 5.3595e-04 - accuracy: 1.0000 - val_loss: 0.4643 - val_accuracy: 0.7188
Epoch 8/8
9/9 [==================

In [42]:
from keras.models import load_model

In [39]:
model.save('facerecog_model.h5')